In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [27]:
def resize(img):
    img= cv2.resize(img, (960,540), interpolation = cv2.INTER_AREA)
    gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    return gray_img

In [34]:
def cropping(gray_img):
    pts = np.array([[
        [100, 550],
        [930, 550],
        [510, 315],
        [440, 315],
    ]],np.int32)

    pts = pts.reshape((-1, 1, 2))
    
    isClosed = True
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.polylines() method
    # Draw a Blue polygon with 
    # thickness of 1 px
    cropped = cv2.polylines(gray_img, [pts], 
                        isClosed, color, thickness)

    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    cropped = cropped[y:y+h, x:x+w].copy()

    # make mask
    pts = pts - pts.min(axis=0)

    mask = np.zeros(cropped.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

    ## do bit-op
    dst = cv2.bitwise_and(cropped, cropped, mask=mask)
    return dst

In [38]:
def threshhold(dst):
    #Blur

    blur = cv2.blur(dst,(5,5),1)
    #Threshold

    _,thresh = cv2.threshold(blur,190,255,cv2.THRESH_BINARY)

    #Opening 

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,10))
    opening = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)

    #Erosion

    kernel = np.ones([2,12])
    erosion = cv2.erode(opening,kernel)
    return erosion


In [40]:
def drawing_lines(img,erosion):
    lines = cv2.HoughLines(erosion, 1, np.pi / 180, 50)
    if type(lines) == np.ndarray :
            for line in lines:
                for r, theta in line:
                    
                    # Stores the value of cos(theta) in a
                    a = np.cos(theta)

                    # Stores the value of sin(theta) in b
                    b = np.sin(theta)

                    # x0 stores the value rcos(theta)
                    x0 = a * r

                    # y0 stores the value rsin(theta)
                    y0 = b * r

                    # x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
                    x1 = int(x0 + 1000 * (-b))

                    # y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
                    y1 = int(y0 + 1000 * (a))

                    # x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
                    x2 = int(x0 - 1000 * (-b))

                    # y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
                    y2 = int(y0 - 1000 * (a))

                    # cv2.line draws a line in img from the point(x1,y1) to (x2,y2).
                    # (0,0,255) denotes the colour of the line to be
                    # drawn. In this case, it is red.

                    


                    if abs(a)>0.51 and 0.52>abs(a) or a>0.1:
                        cv2.line(img[y:y+h, x:x+w], (x1, y1), (x2, y2), (0, 255, 0), 2)
                    elif abs(a)<0.1:
                        pass
                    else : 
                        pass
            else:
                pass
    return img


In [42]:
capture = cv2.VideoCapture("Test/test_videos/solidWhiteRight.mp4")
window_name="challenge"
if not capture.isOpened():
    exit(0)
while True:
    ret,img = capture.read()
    if img is None:
        break

    gray_img =resize(img)
    """ Cropping """
    dst = cropping(gray_img)

    """ blur, threshold operations"""

    erosion=threshhold(dst)

    """Finding and drawing lines"""

    img = drawing_lines(img,erosion)
    
    
    
    
    
    cv2.imshow(window_name,img)


    k = cv2.waitKey(30)
    if k == ord("q"):
        break
capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)



-1